This code extracts data from the wind_data files to calculate the mean and maximum in-flight windspeed for each video clip. These variables are then stored in the env_variables_clip2.csv file.

In [ ]:
# import packages
import pandas as pd
import numpy as np
import os
import glob

os.getcwd()

In [ ]:
# set folders
save_folder = "/Users/blaircostelloe/Dropbox/Publications/DroneDisturbance/drone-disturbance/supplement/processed-data/"
